In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from random import randint
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import random

In [3]:
tags_animes = pd.read_csv(os.path.join("data", "tags_animes.csv"))

In [9]:
detalles_animes = pd.read_csv(os.path.join("data", "MAL_dataframe.csv"))

In [4]:
# Convertir los posibles valores NaN de Tags a "" (caso hipótetico)

tags_animes["Tags"] = tags_animes["Tags"].fillna("")

In [5]:
# Será conveniente renombrar MAL_ID a id para poder hacer un merge con el dataframe de
# detalles_animes

tags_animes = tags_animes.rename(columns={"MAL_ID": "id"})

In [6]:
# Obtener todos los tags únicos en el DataFrame

all_tags = set(
    tag.strip() if isinstance(tags_str, str) else tag
    for tags_str in tags_animes["Tags"]
    for tag in (
        tags_str.strip("[]").split(",") if isinstance(tags_str, str) else tags_str
    )
)


# Además, quitar las ' de los tags, que no sirven de nada, y están al comienzo y al final de cada
# tag

all_tags = set(tag.strip("'") for tag in all_tags)

# Eliminar el tag fantasma de all_tags, cuyo valor es ""; no sirve de nada la verdad.
try:
    all_tags.remove("")
except KeyError:
    pass
try:
    all_tags.remove(" ")
except KeyError:
    pass
try:
    all_tags.remove("")
except KeyError:
    pass
try:
    all_tags.remove(" ")
except KeyError:
    pass
try:
    all_tags.remove("  ")
except KeyError:
    pass

# Crear columnas para cada tag en el DataFrame
for tag in all_tags:
    tags_animes[tag] = tags_animes["Tags"].apply(lambda x: 1 if tag in x else 0)

C:\Users\mativ\AppData\Local\Temp\ipykernel_36576\4075139387.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tags_animes[tag] = tags_animes["Tags"].apply(lambda x: 1 if tag in x else 0)
C:\Users\mativ\AppData\Local\Temp\ipykernel_36576\4075139387.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tags_animes[tag] = tags_animes["Tags"].apply(lambda x: 1 if tag in x else 0)
C:\Users\mativ\AppData\Local\Temp\ipykernel_36576\4075139387.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of c

In [7]:
# Por conveniencia, lo más útil es eliminar la columna "URL", junto a "Tags" y "Alts", que ya no
# sirven de nada. Además se establece la columna "MAL_ID" como índice.

tags_animes = tags_animes.drop(columns=["URL", "Tags", "Alts", "Anime"]).set_index("id")

In [10]:
# El MAL_ID de índice puede conectarse con los datos de detalles_animes, que también tienen MAL_ID,
# y así se puede obtener que el índice sea el title de cada anime, que es más útil y fácil de
# visualizar.

tags_animes = tags_animes.join(detalles_animes.set_index("id")["title"])

In [11]:
# Cambio de índice a title, eliminación del índice actual igualmente.

tags_animes = tags_animes.set_index("title")

In [12]:
# Por temas de memoria principalmente, es conveniente hacer el dataframe sparse.

tags_animes = tags_animes.astype(pd.SparseDtype(np.int8, fill_value=0))

In [13]:
tags_animes 

,Monster Girls,Family Friendly,Borderline H,Dodgeball,Yuri,Bathhouse,Cyberpunk,Recap,Running,Witches,...,9th Century,Psychic Powers,Natural Disaster,Mature Themes,Romantic Comedy,Shoujo,Robots,Dragon Quest,Ancient China,Parody
title,,,,,,,,,,,,,,,,,,,,,
"""0""",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""Bungaku Shoujo"" Memoire",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""Bungaku Shoujo"" Movie",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""Eiji""",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zuori Qing Kong,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Zurui Maboroshi,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Zutto Mae kara Suki deshita. Kokuhaku Jikkou Iinkai,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Dado que tenemos demasiadas columnas de información, lo que haremos sea lo siguiente para unificar
todo para el clustering. 

En primer lugar, 